Reference

- https://github.com/adapter-hub/adapters
- https://docs.adapterhub.ml/loading.html
- https://docs.adapterhub.ml/adapter_composition.html


-https://huggingface.co/docs/transformers/peft

In [1]:
!sudo -H pip install -Uq transformers

In [20]:
!sudo -H pip uninstall accelerate -y

In [1]:
import accelerate

In [2]:
accelerate.__version__

'0.25.0'

In [22]:
!sudo pip install -Uq accelerate

     |████████████████████████████████| 266kB 6.6MB/s eta 0:00:01


In [1]:
import os
import torch
from datasets import load_dataset
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
import pandas as pd
import torch

/opt/conda/lib/python3.8/site-packages/transformers/utils/hub.py:123: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = True


# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)


model_name = "mistralai/Mistral-7B-Instruct-v0.1"

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="cuda:0",
#     load_in_4bit=True,
#     torch_dtype=torch.float16
)
model.config.use_cache = False
model.config.pretraining_tp = 1

OSError: [Errno 30] Read-only file system: '/data/huggingface/cache/models--mistralai--Mistral-7B-Instruct-v0.1/snapshots/9ab9e76e2b09f9f29ea2d56aa5bd139e4445c59e'

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [5]:
#!sudo -H pip install peft

In [7]:
from peft import PeftModel, PeftConfig

In [8]:
adapter1 = "/data/mistral/query-to-mql/exp-10/nov-20/checkpoint-8000"
adapter2 = "/data/mistral/query-to-mql/exp-10/nov-20/checkpoint-4000"

In [9]:
# using load_adapter

model.load_adapter(adapter1, adapter_name = 'adapter1')


In [10]:
%%time
model.load_adapter(adapter2, adapter_name = 'adapter2')

CPU times: user 1.64 s, sys: 11.3 ms, total: 1.65 s
Wall time: 576 ms


In [11]:
%%time

model.set_adapter('adapter2')

CPU times: user 3.84 ms, sys: 0 ns, total: 3.84 ms
Wall time: 3.48 ms


In [29]:
def predict_template_query_v1(user_query):
    inp = query_template_v1.format(context=context,
                                   user_query=user_query,
                                  date_input=date_input)
    _inputs = tokenizer.encode(inp, return_tensors="pt").to('cuda')
    outputs = model.generate(input_ids=_inputs, max_length= 1000, pad_token_id=tokenizer.eos_token_id)
    output = tokenizer.decode(outputs[0])
    return output
#     output_new = output.split('[MQL]\n')[1]
#     return output_new.split('\n[/MQL]')[0], output

In [30]:
import time

In [47]:
model.set_adapter('adapter1')
start = time.time()
user_query = 'what is purchase across segments'
print('user query: ', user_query)
print('-'*100)
output = predict_template_query_v1(user_query=user_query)
print("time taken : ", time.time()-start)

user query:  what is purchase across segments
----------------------------------------------------------------------------------------------------
time taken :  220.52627992630005


In [48]:
output.split('[MQL]\n')[1].split('\n[/MQL]')[0]

"{'DIMENSION': {'segments': [{'ENTITY': 'Segment', 'RANK': [{'RANK ADJECTIVE': '', 'RANK VALUE': ''}]}]}, 'FILTER': {}, 'DERIVED MEASURE': {}, 'DATE VARIABLE': {'across': [{'ENTITY': 'Order Date', 'DATE RANGE': '01/01/2020 - 15/09/2023'}]}}"

In [49]:
model.set_adapter('adapter2')
start = time.time()
user_query = 'what is purchase across segments'
print('user query: ', user_query)
print('-'*100)
output = predict_template_query_v1(user_query=user_query)
print("time taken : ", time.time()-start)

user query:  what is purchase across segments
----------------------------------------------------------------------------------------------------
time taken :  222.86452674865723


In [50]:
output.split('[MQL]\n')[1].split('\n[/MQL]')[0]

"{'DIMENSION': {'segments': [{'ENTITY': 'Segment', 'RANK': [{'RANK ADJECTIVE': '', 'RANK VALUE': ''}]}]}, 'MEASURE': {'purchase': [{'ENTITY': 'Purchase Vol', 'MEASURE CONSTRAINT': [{'COMPARISON VALUE': '', 'COMPARSION OPERATOR': ''}]}]}}"

In [15]:
context = """{
    "MEASURE": [{"ENTITY": "Discount", "other names": ["discount", "discount rate", "discount value", "deduction"]},
                {"ENTITY": "Purchase Vol", "other names": ["purchase", "purchase value", "purchase model"]},
                {"ENTITY": "Quantity", "other names": ["quantity", "volume"]},
                {"ENTITY": "Sales", "other names": ["sales", "sale"]}],
    "DIMENSION": [{"ENTITY": "Sub-Category", "other names": ["sub-category", "sub category", "categories", "section"]},
                  {"ENTITY": "Segment", "other names": ["segment", "segments", "units", "divisions"]},
                  {"ENTITY": "Parts", "other names": ["parts", "part", "section", "divisions"]},
                  {"ENTITY": "Country", "other names": ["country", "countries"]}],
    "FILTER": [{"ENTITY": "Consumer", "other names": ["consumers", "consumer"], "parent": "Segment"},
               {"ENTITY": "Phone", "other names": ["phone", "phones", "mobile phones"], "parent": "Sub-Category"},
               {"ENTITY": "Binder", "other names": ["binders", "binder"], "parent": "Sub-Category"},
               {"ENTITY": "Corporate", "other names": ["corporates", "corporate"], "parent": "Segment"},
               {"ENTITY": "India", "other names": ["india"], "parent": "Country"},
               {"ENTITY": "Dubai", "other names": ["dubai"], "parent": "Country"}],
    "DERIVED MEASURE": [{"ENTITY": "Ratio",
             "other names": ["ratio", "share", "contribution", "percentage", "proportion", "contributing"]},
            {"ENTITY": "Why", "other names": ["why", "cause of", "reason for", "diagnose"]},
            {"ENTITY": "contribution_to_growth", "other names": ["contribution to growth", "growth", "grown"]},
            {"ENTITY": "kda_transactional", "other names": ["kda", "key drivers", "key driver", "drivers", "driver"]},
            {"ENTITY": "Growth Rate", "other names": ["growth rate", "growth", "grown"]},
            {"ENTITY": "correlation",
             "other names": ["associate", "associated", "association", "associations", "correlate", "correlated",
                             "correlation", "correlations", "relate", "related", "relation", "relations",
                             "relationship",
                             "relationships"]}
            ],
    "DATE VARIABLE": [{"ENTITY": "Order Date", "other names": ["order date", "date", "trend", "time", "when", "mom", "yoy"]}]
    }"""

date_input = {
    "start_date": "01/01/2020",
    "end_date": "15/09/2023"
}

In [16]:
query_template_v1 = """Given the context : {context} and date reference: {date_input}, the query: {user_query}, is converted into below shown structured output.
[MQL]
"""

In [ ]:
# model = AutoAdapterModel.from_pretrained("roberta-base")


model.load_adapter("AdapterHub/roberta-base-pf-imdb", source="hf", set_active=True)

print(model(**tokenizer("This works great!", return_tensors="pt")).logits)